# [SETUP] 
connect to DuckDB

In [1]:
! python ./setup.py

Cleaning up (if any existing) tpch db file
Creating TPCH input data


In [2]:
import duckdb
import pandas as pd

%load_ext sql
conn = duckdb.connect("tpch.db")
%sql conn --alias duckdb

In [3]:
%%sql
show tables;

Running query in 'duckdb'

name
customer
lineitem
nation
orders
part
partsupp
region
supplier
